<a href="https://colab.research.google.com/github/JacopoMangiavacchi/animegan2-coreml/blob/master/convert_coreml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install coremltools

In [2]:
import os
import torch
import coremltools

In [3]:
!git clone https://github.com/jacopomangiavacchi/animegan2-coreml

Cloning into 'animegan2-coreml'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 67 (delta 24), reused 36 (delta 6), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [4]:
os.chdir(f'./animegan2-coreml')

In [5]:
from model import Generator

device = 'cpu'
torch.set_grad_enabled(False)

model = Generator().eval().to(device)

In [6]:
ckpt = torch.load(f"weights/celeba_distill.pt", map_location=device)
# ckpt = torch.load(f"weights/face_paint_512_v2.pt", map_location=device)
# ckpt = torch.load(f"weights/paprika.pt", map_location=device)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [7]:
image_sizes = [256, 512, 1024]

for image_size in image_sizes:
    image = torch.randn(1, 3, image_size, image_size)
    output = model(image.to(device))

    scripted_model = torch.jit.script(model)
    # traced = torch.jit.trace(model, image)

    image_input = coremltools.ImageType(name="input_1", shape=(1, 3, image_size, image_size))

    mlmodel = coremltools.converters.convert(
      scripted_model,
      inputs=[image_input],
    )

    mlmodel.save(f'animegan2-celeba-distil-{image_size}.mlmodel')    
    spec = coremltools.utils.load_spec(f"animegan2-celeba-distil-{image_size}.mlmodel")

    output = spec.description.output[0]

    output.type.imageType.colorSpace = coremltools.proto.FeatureTypes_pb2.ImageFeatureType.RGB
    output.type.imageType.height = image_size
    output.type.imageType.width = image_size

    coremltools.utils.save_spec(spec, f"animegan2-celeba-distil-{image_size}.mlmodel")

AssertionError: ignored

In [ ]:
os.chdir(f'..')